In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [ ]:
# Setting random seed for reproducibility
torch.manual_seed(140)
np.random.seed(140)
random.seed(140)

In [ ]:
# Simulate data 
x_values = torch.tensor([1, 2, 3, 4], dtype=torch.float32).view(-1, 1, 1)
y_values = torch.tensor([10, 20, 30, 40], dtype=torch.float32).view(-1, 1, 1)

# Create a scaler
sc_x = MinMaxScaler()
sc_y = MinMaxScaler()

# Shape the data from 3D to 2D
x_values_2D = x_values.squeeze(-1).numpy()
y_values_2D = y_values.squeeze(-1).numpy()

# Fit the scaler and transform the data
x_scaled = sc_x.fit_transform(x_values_2D)
y_scaled = sc_y.fit_transform(y_values_2D)

# Reshape the data back to 3D and convert to tensor
x_scaled = torch.tensor(x_scaled).unsqueeze(-1)
y_scaled = torch.tensor(y_scaled).unsqueeze(-1)

In [ ]:
# Define the LSTM model
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :]) # Extract only the last timestep's output for prediction
        return x

In [ ]:
def train_model(x_scaled, y_scaled):
    num_epochs = 50
    learning_rate = 0.05

    input_size = 1
    hidden_size = 10
    num_layers = 1

    num_classes = 1

    # Initialize the model
    model = SimpleLSTM(input_size, hidden_size, num_classes, num_layers)
    # Initialize the optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss() # Mean Squared Error Loss

    # Train the model
    for epoch in range(num_epochs):
        model.train() # Set the model to training mode
        optimizer.zero_grad() # Clear the gradients
        output = model(x_scaled) # Forward pass
        loss = loss_fn(output, y_scaled.view(-1, 1)) # Compute the loss
        loss.backward() # Backward pass
        optimizer.step() # Update the weights
        
        if epoch % 5 == 0: # Print the loss every 5 epochs
            print(f'Epoch [{epoch}/{num_epochs}], Loss: {loss.item()}')

    return model

In [ ]:
model = train_model(x_scaled, y_scaled) # Train the model

# Your manually specified actual_y values
actual_y = np.array([10, 20, 30, 40, 50])  # Example actual y values for x values [1, 2, 3, 4]

# Assuming `sc_x` and `sc_y` are already fitted to your training data
# Prepare multiple x values for prediction
x_to_predict = torch.tensor([1, 2, 3, 4, 5], dtype=torch.float32).view(-1, 1, 1)  # Example x values
x_to_predict_2D = x_to_predict.squeeze(-1).numpy()  # Convert to 2D for scaling
x_to_predict_scaled = sc_x.transform(x_to_predict_2D)  # Scale x values
x_to_predict_scaled_tensor = torch.tensor(x_to_predict_scaled, dtype=torch.float32).view(-1, 1, 1)  # Convert back to tensor

model.eval()    # Set the model to evaluation mode
predictions_scaled = [] # Store the scaled predictions
with torch.no_grad():  # Disable gradient tracking
    for x_val in x_to_predict_scaled_tensor: # Iterate over each x value
        prediction_scaled = model(x_val.unsqueeze(0))  # Add batch dimension
        predictions_scaled.append(prediction_scaled) # Store the scaled prediction

# Convert predictions to a numpy array for inverse transform
predictions_scaled_np = torch.cat(predictions_scaled, dim=0).numpy()
predictions = sc_y.inverse_transform(predictions_scaled_np.reshape(-1, 1))  # Inverse transform predictions

# Print Actual vs Predicted
predicted_y = predictions.flatten()

# Print the actual and predicted values
print("X Value\tActual Y\tPredicted Y")
for i, x_val in enumerate(x_to_predict.numpy().flatten()):
    print(f"{x_val}\t{actual_y[i]}\t{predicted_y[i]}")

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(x_to_predict.numpy().flatten(), actual_y, 'bo-', label='Actual')
plt.plot(x_to_predict.numpy().flatten(), predicted_y, 'ro-', label='Predicted')
plt.title('LSTM Predictions vs Actual')
plt.xlabel('X Value')
plt.ylabel('Y Value')
plt.legend()
plt.show()